# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [17]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [18]:
"""
The lines below for:
  checking your current working directory
  Get your current folder and subfolder event data
  Create a for loop to create a list of files and collect each filepath
  join the file path and roots with the subdirectories using glob
"""
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [19]:
"""
The script below does:
 Initiating an empty list of rows that will be generated from each file, for every filepath in the file path list, reads csv file
 Creates a csv reader object, extract each data row one by one and append it.
 Create a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
 
"""

full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [20]:
"""
  Checking the number of rows in your csv file and printing the sum.
"""
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the after the code above is run
<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [21]:
"""
  This should make a connection to a Cassandra instance your local machine
  (127.0.0.1) to establish connection and begin executing queries, for a expecifique session.
  
"""
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [22]:
"""
 Creating a key space in Cassandra.
 A keyspace is like a table repository.
"""
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS events
        WITH REPLICATION = { 'class':'SimpleStrategy', 'replication_factor':1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [23]:
"""
  Setting up a  KEYSPACE to the keyspace specified above.
"""
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [24]:
"""
 this script will create a table song_sessions in way that we can retreave the  song title and song's length in the music app history that was heard during 
 sessionId = 338, and itemInSession = 4
"""  
query1_create = 'CREATE TABLE IF NOT EXISTS song_sessions'
query1_create = query1_create + '(session_id int, item_in_session int, artist_name text, song_title text, song_duration decimal, PRIMARY KEY(session_id, item_in_session))'

try:
    session.execute(query1_create)
except Exception as e:
    print(e)       

In [25]:
"""
 the code below to set up the CSV file. 
 use a csv.reader object to walktroutgh al file, select the (session_id, item_in_session, artist_name, song_title, song_duration)
 fields into song_sessions table
 
"""
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_sessions(session_id, item_in_session, artist_name, song_title, song_duration)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

#### Do a SELECT to verify that the data have been inserted into each table

In [29]:
"""
 The SELECT statement is to verify the data was entered into the table.
"""

query1_select = "SELECT artist_name, song_title, song_duration FROM song_sessions WHERE session_id = 338 and  item_in_session = 4"
try:
    rows = session.execute(query1_select)
except Exceptions as e:
    print(e)
for row in rows:
    print(row.artist_name, row.song_title, row.song_duration )

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [30]:
"""
 this script will create a table song_users in way that we can retreave  the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
 for userid = 10, sessionid = 182
 use a csv.reader object to walktroutgh all file, select the (user_id, session_id, artist_name, song_title, item_in_session, user_firstname, user_lastname)
 fields into song_users table
"""  
query2_create = 'CREATE TABLE IF NOT EXISTS song_users'
query2_create = query2_create + '(user_id int, session_id int, artist_name text, song_title text, item_in_session int, user_firstname text, user_lastname text \
    , PRIMARY KEY( (user_id, session_id), item_in_session, user_firstname, user_lastname) )'
try:
    session.execute(query2_create)
except Exception as e:
    print(e)
                
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_users(user_id, session_id, artist_name, song_title, item_in_session, user_firstname, user_lastname)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4] ))                    

In [31]:
"""
 The SELECT statement is to verify the data was entered into the table.
"""
query1_select = "SELECT artist_name, song_title, song_title, item_in_session, user_firstname, user_lastname FROM song_users WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query1_select)
except Exceptions as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_title, row.item_in_session, row.user_firstname, row.user_lastname  )                   

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


In [32]:
 """
  this script will create a table song_title in way that we can retreave  the following: 
  every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own
  use a csv.reader object to walktroutgh all file, select the (song_title, user_firstname, user_lastname) 
  fields into song_title table.
 
 """
query3_create = 'CREATE TABLE IF NOT EXISTS song_title'
query3_create = query3_create + '(song_title text, user_firstname text, user_lastname text \
    , PRIMARY KEY( (song_title), user_firstname, user_lastname) )'

try:
    session.execute(query3_create)
except Exception as e:
    print(e)
                
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_title(song_title, user_firstname, user_lastname)"
        query = query + "VALUES(%s, %s, %s)"
        session.execute(query, ( line[9], line[1], line[4] ))

In [33]:
"""
 The SELECT statement is to verify the data was entered into the table.
"""
query3_select = "SELECT song_title, user_firstname, user_lastname FROM song_title WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query3_select)
except Exceptions as e:
    print(e)

for row in rows:
    print(row.song_title, row.user_firstname, row.user_lastname  )

All Hands Against His Own Jacqueline Lynch
All Hands Against His Own Sara Johnson
All Hands Against His Own Tegan Levine


### Drop the tables before closing out the sessions

In [34]:
"""
 This script drops the table before closing out the sessions.
"""
drop_query1 = "DROP TABLE song_sessions"
drop_query2 = "DROP TABLE song_users"
drop_query3 = "DROP TABLE song_title"

try:
    rows1 = session.execute(drop_query1)
    rows2 = session.execute(drop_query2)
    rows3 = session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()